In [1]:
# Predict Activity from No Action Elements
# - are there attributes that are unique for each activity?

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../../data_v3/hr_extended_features.csv')

In [4]:
labels = df["BusinessActivity"].values

In [5]:
features = df[["event_with_roles","request_method_call", "selective_filter_data"]].fillna("")

In [3]:
df_gt = pd.read_csv('../../data_v3/hr_ground_truth.csv')

In [4]:
# Chain Features 5 Events backward 
df_train = pd.read_csv('../../data_v3/hr_extended_features.csv')

# Form Sequences by Grouping
df_train = df_train.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_train["SequenceNumber"] = df_train.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
df_train["SequenceNumber"] -= df_train['SequenceNumber'].min()

# Random Forest

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(features), labels, test_size=0.3, random_state=42)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_9273/3279782877.py", line 3, in <module>
    X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(features), labels, test_size=0.3, random_state=42)
NameError: name 'features' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.struc

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
clf = RandomForestClassifier(random_state=42)

In [7]:
clf.fit(X_train, y_train)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_9273/533357445.py", line 1, in <module>
    clf.fit(X_train, y_train)
NameError: name 'clf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site

In [8]:
from sklearn.metrics import classification_report
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_9273/1983594605.py", line 2, in <module>
    pred = clf.predict(X_test)
NameError: name 'clf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/si

# Sequence Classification Using CRF

In [9]:
# Divides dataframe into arrays according to to Sequence Data Indicator
def sequence_by_activities(data, seq_data): 
    return [data[seq_data == i].values for i in range(seq_data.max())]


cols = ["event_with_roles","request_method_call","selective_filter_data","origin_selective_filter_data","BusinessActivity"]

train_data = sequence_by_activities(df_train[cols], df_train["SequenceNumber"])
train_labels = [seq[:,-1].tolist() for seq in train_data]

In [10]:
def seq2features(seq, bw, fw ,feature_list): 
    # seq [0] = Position in Sequence
    # seq [0,4] = Feature 4 an Position 0 
    return [event2features(seq,i, bw, fw, feature_list) for i in range(len(seq))]


def event2features(seq, i,bw, fw, feature_list): 
    
    features = {"bias": 1.0}
    # event features 
    features.update( {
        f"0:{feature_list[j]}": seq[i,j] for j in range(len(feature_list))        
    })
    
    # backward features 1...bw
    for k in range(1,bw+1): 
        index = i - k
        if index >= 0: 
            features.update({
                f"-{k}:{feature_list[j]}" : seq[index,j] for j in range(len(feature_list))
            })
        else: 
             features.update({
                f"-{k}:{feature_list[j]}" : "NoMessage" for j in range(len(feature_list))
            })
              
    # forward features 1...fw
    for k in range(1, fw+1): 
        index = i + k
        if index < len(seq): 
            features.update({
                f"+{k}:{feature_list[j]}" : seq[index, j] for j in range(len(feature_list))
            })
        else: 
            features.update({
                f"+{k}:{feature_list[j]}": "NoMessage" for j in range(len(feature_list))
            })
            
    # remove all empty String features        
    #features = {key: value for key, value in features.items() if value != ''}
    return features

In [11]:
features = ["event_with_roles","request_method_call","selective_filter_data","origin_selective_filter_data"]
bw = 5
fw = 0
train_features = [seq2features(s, bw,fw, features) for s in train_data]
#test_features = [seq2features(s, bw,fw,features) for s in test_data]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.5, random_state=42, shuffle=True)

In [13]:
%%time
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=200, 
    #all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 1.44 s, sys: 36.5 ms, total: 1.48 s
Wall time: 1.75 s


CRF(algorithm='lbfgs', max_iterations=200)

In [14]:
from sklearn_crfsuite import metrics
from sklearn.metrics import multilabel_confusion_matrix
def flatten(xss): 
    return [x for xs in xss for x in xs]

def evaluate(model, x, y_true): 
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels = model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    print(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred)))
    
#evaluate(crf, X_test, y_test)

In [15]:
y_pred = crf.predict_single(flatten(X_test))
print(classification_report(flatten(y_test), y_pred))

                                 precision    recall  f1-score   support

               ContractProposal       1.00      0.86      0.93       897
 GenerateJobApplicationActivity       0.89      0.89      0.89      1890
         PerformAnInterviewCall       0.93      0.75      0.83      1344
      PerformAnInterviewMeeting       1.00      0.98      0.99      1385
           ResumeReviewActivity       0.91      1.00      0.95      5925
ScheduleAnInterviewActivityCall       0.83      0.88      0.86      2056
     ScheduleAnInterviewMeeting       0.92      0.83      0.87      2492

                       accuracy                           0.91     15989
                      macro avg       0.93      0.88      0.90     15989
                   weighted avg       0.92      0.91      0.91     15989


In [30]:
# Model that classifies the Activity for single events in a sequence
from joblib import dump, load
dump(crf, "activity_model_v1.joblib")

['activity_model_v1.joblib']

In [31]:
# TODO Test with Random Forest or sth. 

In [32]:
%%time
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=200, 
    #all_possible_transitions=True
)
crf.fit([flatten(X_train)], [flatten(y_train)])

CPU times: user 1.57 s, sys: 25.8 ms, total: 1.6 s
Wall time: 1.81 s


CRF(algorithm='lbfgs', max_iterations=200)

In [33]:
y_pred = crf.predict_single([x for xs in X_test for x in xs])
print(classification_report([x for xs in y_test for x in xs], y_pred))

                                 precision    recall  f1-score   support

               ContractProposal       0.95      0.94      0.95       897
 GenerateJobApplicationActivity       0.88      1.00      0.94      1890
         PerformAnInterviewCall       0.94      0.89      0.92      1344
      PerformAnInterviewMeeting       1.00      1.00      1.00      1385
           ResumeReviewActivity       0.99      0.97      0.98      5925
ScheduleAnInterviewActivityCall       0.85      0.99      0.92      2056
     ScheduleAnInterviewMeeting       1.00      0.83      0.91      2492

                       accuracy                           0.95     15989
                      macro avg       0.95      0.95      0.94     15989
                   weighted avg       0.95      0.95      0.95     15989


In [34]:
# Trained on single activities
from joblib import dump, load
dump(crf, "activity_model_v2.joblib")

['activity_model_v2.joblib']

# Next Event Estimation using Markov Chains

In [10]:
#from pomegranate.markov_chain import MarkovChain
import torch

In [13]:
cols = ["event_with_roles","selective_filter_data","origin_selective_filter_data"]
seq_numbers = df_train["SequenceNumber"]
df = df_train[cols].fillna("")

In [14]:
df["joined"] = pd.DataFrame({"col":df["event_with_roles"] + df["selective_filter_data"]})

In [15]:
from sklearn.preprocessing import LabelEncoder

# Label Encode DF 
le1 = LabelEncoder()
le2 = LabelEncoder()
df["event_with_roles_LE"] = le1.fit_transform(df["event_with_roles"])
df["selective_filter_data_LE"] = le2.fit_transform(df["selective_filter_data"])
df["origin_selective_filter_data_LE"] = le2.transform(df["origin_selective_filter_data"])

In [16]:
le3 = LabelEncoder()
df["joined_LE"] = le3.fit_transform(df["joined"])

In [17]:
# Divides dataframe into arrays according to to Sequence Data Indicator
def sequence_by_activities(data, seq_data): 
    return [data[seq_data == i].values for i in range(seq_data.max())]

cols_LE = ["event_with_roles_LE","selective_filter_data_LE","origin_selective_filter_data_LE"]
cols = ["event_with_roles","selective_filter_data","origin_selective_filter_data"]

data_LE = sequence_by_activities(df[cols_LE], df_train["SequenceNumber"])

data_joined_LE = sequence_by_activities(df["joined_LE"], df_train["SequenceNumber"])

data_joined = sequence_by_activities(df["joined"], df_train["SequenceNumber"])

In [55]:
from torch.nn.utils.rnn import pad_sequence

tensor = pad_sequence([torch.tensor(x) for x in data], batch_first=True, padding_value=False)
tensor = tensor.int()
tensor = tensor.unsqueeze(2)
tensor = torch.unique(tensor, dim = 0)

In [31]:
from pomegranate.markov_chain import MarkovChain

model = MarkovChain(k=3)

X = torch.tensor([[[1], [0], [0], [1]],
                  [[0], [1], [0], [0]],
                  [[0], [0], [0], [0]],
                  [[0], [0], [0], [1]],
                  [[0], [1], [1], [0]]])

model.fit(X)

MarkovChain()

# Pomegranate 0.14.8

In [26]:
from pomegranate import *

In [22]:
import numpy as np

# Convert each array to a tuple and create a set of tuples
array_set = set(tuple(arr) for arr in data_joined)

# Convert the set of tuples back to a list of NumPy arrays
unique_array_list = [np.array(arr) for arr in array_set]

len(unique_array_list)

26

In [23]:
mc = MarkovChain.from_samples(unique_array_list, k=3)

In [24]:
with open("markov_v2.json", "w") as outfile:
    json.dump(mc.to_json(), outfile)

In [25]:
data_joined[0][:5]

array(['End Point (HR Manager)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n]version',
       'Odoo Application->End Point (HR Manager): [HttpResponse:HTTP/1.0 200 OK\\r\\n]',
       'End Point (HR Manager)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n]',
       "Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:INSERT:['res_users_log']]",
       'Odoo Application->End Point (HR Manager): [HttpResponse:HTTP/1.0 200 OK\\r\\n]IsNumber'],
      dtype=object)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 0 dimension(s)